<a href="https://colab.research.google.com/github/escorpioh/hacking-civico/blob/master/tareas/03-exploracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Para descargar el conjunto de datos
!wget http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

# Para descargar el diccionario de los datos
!wget http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip

# Descomprimimos ambas carpetas y movemos archivos
!unzip datos_abiertos_covid19.zip
!unzip diccionario_datos_covid19.zip
!mv ./diccionario_datos_covid19/*.xlsx .
!mv *COVID19MEXICO.csv COVID19MEXICO.csv

# Eliminamos las carpetas comprimidas
!rm -rf diccionario_datos_covid19
!rm datos_abiertos_covid19.zip
!rm diccionario_datos_covid19.zip

--2020-09-20 23:28:50--  http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 187.210.186.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|187.210.186.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33390212 (32M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip’

datos_abiertos_covi 100%[===================>]  31.84M  20.8MB/s    in 1.5s    

2020-09-20 23:28:52 (20.8 MB/s) - ‘datos_abiertos_covid19.zip’ saved [33390212/33390212]

--2020-09-20 23:28:52--  http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip
Resolving epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)... 187.191.75.207
Connecting to epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)|187.191.75.207|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ep

In [45]:
import statistics

import pandas as pd
import numpy as np

In [72]:
# Especificamos el nombre del archivo por abrir
nombre_archivo = 'COVID19MEXICO.csv'

# Utilizamos la función de Pandas para cargar un archivo CSV
data = pd.read_csv(nombre_archivo, encoding='latin-1')

In [73]:
data.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-09-19,13f301,2,3,15,2,9,15,106,2,2020-05-27,2020-05-25,9999-99-99,2,2,54,1,97,2,2,2,2,2,2,2,2,2,2,2,2,1,99,MÃ©xico,99,2
1,2020-09-19,0e7edf,2,3,27,2,27,27,4,1,2020-06-18,2020-06-08,9999-99-99,97,2,31,1,97,2,2,2,2,2,2,2,2,2,2,1,1,1,99,MÃ©xico,99,97
2,2020-09-19,10d9be,2,4,2,2,10,2,4,2,2020-04-06,2020-03-28,2020-04-12,1,1,43,1,97,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,2
3,2020-09-19,11a9b3,2,4,9,2,9,15,58,1,2020-04-20,2020-04-20,9999-99-99,97,2,32,1,97,2,2,2,2,2,1,2,2,2,2,2,1,1,99,MÃ©xico,99,97
4,2020-09-19,077c96,2,4,9,2,16,9,5,2,2020-04-30,2020-04-30,2020-05-15,1,1,66,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,1


In [74]:
import altair as alt
import seaborn as sns


In [75]:
nombre_diccionario = 'Descriptores_0419.xlsx'
diccionario = pd.read_excel(nombre_diccionario)

In [76]:
diccionario

,Nº,NOMBRE DE VARIABLE,DESCRIPCIÓN DE VARIABLE,FORMATO O FUENTE
0,1,FECHA_ACTUALIZACION,"La base de datos se alimenta diariamente, esta...",AAAA-MM-DD
1,2,ID_REGISTRO,Número identificador del caso,TEXTO
2,3,ORIGEN,La vigilancia centinela se realiza a través de...,CATÁLOGO: ORIGEN ...
3,4,SECTOR,Identifica el tipo de institución del Sistema ...,CATÁLOGO: SECTOR ...
4,5,ENTIDAD_UM,Identifica la entidad donde se ubica la unidad...,CATALÓGO: ENTIDADES
5,6,SEXO,Identifica al sexo del paciente.,CATÁLOGO: SEXO
6,7,ENTIDAD_NAC,Identifica la entidad de nacimiento del paciente.,CATALÓGO: ENTIDADES
7,8,ENTIDAD_RES,Identifica la entidad de residencia del paciente.,CATALÓGO: ENTIDADES
8,9,MUNICIPIO_RES,Identifica el municipio de residencia del paci...,CATALÓGO: MUNICIPIOS
9,10,TIPO_PACIENTE,Identifica el tipo de atención que recibió el ...,CATÁLOGO: TIPO_PACIENTE


In [77]:
nombre_cat = 'Catalogos_0412.xlsx'
sheets = ['ORIGEN', 'SECTOR', 'SEXO', 'TIPO_PACIENTE', 'SI_NO', 'NACIONALIDAD', 'RESULTADO', 'de ENTIDADES', 'MUNICIPIOS']
catalogues = {}
for sheet in sheets:
  catalogue = pd.read_excel(nombre_cat, sheet_name='Catálogo ' + sheet)
  catalogue.columns = catalogue.columns.str.replace(' ', '_')
  catalogue.columns = catalogue.columns.str.replace('.', '')
  catalogue.columns = catalogue.columns.str.lower()
  new_catalogue = {sheet: catalogue}
  catalogues.update(new_catalogue)

In [78]:
catalogues


{'MUNICIPIOS':       clave_municipio        municipio  clave_entidad
 0                   1   AGUASCALIENTES              1
 1                   2         ASIENTOS              1
 2                   3         CALVILLO              1
 3                   4            COSÍO              1
 4                   5      JESÚS MARÍA              1
 ...               ...              ...            ...
 2496              999  NO ESPECIFICADO             32
 2497              999  NO ESPECIFICADO             36
 2498              997        NO APLICA             97
 2499              998        SE IGNORA             98
 2500              999  NO ESPECIFICADO             99
 
 [2501 rows x 3 columns], 'NACIONALIDAD':    clave      descripción
 0      1         MEXICANA
 1      2       EXTRANJERA
 2     99  NO ESPECIFICADO, 'ORIGEN':    clave      descripción
 0      1            USMER
 1      2   FUERA DE USMER
 2     99  NO ESPECIFICADO, 'RESULTADO':   unnamed:_0              unnamed:_1
 0    

In [79]:
catalogues['SEXO']

,clave,descripción
0,1,MUJER
1,2,HOMBRE
2,99,NO ESPECIFICADO


In [80]:
# limpieza del nombre de las columnas, remover espacios, carácteres especiales y pasar a minúsculas
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.lower()

In [81]:
# convertimos a fecha algunas columnas
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

data = cast_datetime_cols(data)

In [82]:
data.head()

,fecha_actualizacion,id_registro,origen,sector,entidad_um,sexo,entidad_nac,entidad_res,municipio_res,tipo_paciente,fecha_ingreso,fecha_sintomas,fecha_def,intubado,neumonia,edad,nacionalidad,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,resultado,migrante,pais_nacionalidad,pais_origen,uci
0,2020-09-19,13f301,2,3,15,2,9,15,106,2,2020-05-27,2020-05-25,NaT,2,2,54,1,97,2,2,2,2,2,2,2,2,2,2,2,2,1,99,MÃ©xico,99,2
1,2020-09-19,0e7edf,2,3,27,2,27,27,4,1,2020-06-18,2020-06-08,NaT,97,2,31,1,97,2,2,2,2,2,2,2,2,2,2,1,1,1,99,MÃ©xico,99,97
2,2020-09-19,10d9be,2,4,2,2,10,2,4,2,2020-04-06,2020-03-28,2020-04-12,1,1,43,1,97,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,2
3,2020-09-19,11a9b3,2,4,9,2,9,15,58,1,2020-04-20,2020-04-20,NaT,97,2,32,1,97,2,2,2,2,2,1,2,2,2,2,2,1,1,99,MÃ©xico,99,97
4,2020-09-19,077c96,2,4,9,2,16,9,5,2,2020-04-30,2020-04-30,2020-05-15,1,1,66,1,97,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,1


In [88]:
# unimos con el catálogo de ENTIDADES
entidades_cols = [col for col in data.columns if "entidad" in col and col != 'entidad_res']
for col in entidades_cols:
    data = (
        data.merge(catalogues['de ENTIDADES'], how='left', left_on=col, right_on='clave_entidad').
        drop(columns=[col, 'abreviatura', 'clave_entidad']).
        rename(columns={'entidad_federativa':col})
    )

In [89]:
# unimos con el catálogo SI_NO
sino_cols = ['intubado', 'neumonia', 'embarazo', 'habla_lengua_indig', 'diabetes', 'epoc', 'asma', 'inmusupr',
             'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 
             'otro_caso', 'migrante', 'uci']
for col in sino_cols:
    data = (
        data.merge(catalogues['SI_NO'], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [91]:
catalogues['RESULTADO'].columns

catalogues['RESULTADO'].columns = ['clave', 'descripción']
catalogues['RESULTADO'].columns

catalogues['RESULTADO'] = catalogues['RESULTADO'].drop(index=0)

catalogues['RESULTADO']['clave'] = catalogues['RESULTADO']['clave'].astype(int)



In [94]:
# unimos otras columnnas que comparten lógica
other_cols = ['origen', 'sector', 'sexo', 'tipo_paciente', 'nacionalidad', 'resultado']
for col, cat in zip(other_cols, [x.upper() for x in other_cols]):
    data = (
        data.merge(catalogues[cat], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [108]:
# creamos algunas variables que podrían ser útiles después
data['defuncion'] = ~data.fecha_def.isna()
data['mes_def'] = data.fecha_def.dt.month
data['año_def'] = data.fecha_def.dt.year
data['mes_ingreso'] = data.fecha_ingreso.dt.month
data['año_ingreso'] = data.fecha_ingreso.dt.year


In [115]:
data.head(3)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso
0,2020-09-19,13f301,2020-05-27,2020-05-25,NaT,54,MÃ©xico,99,MÉXICO,TOLUCA,MÉXICO,CIUDAD DE MÉXICO,NO,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO,FUERA DE USMER,ESTATAL,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,5,2020
1,2020-09-19,0e7edf,2020-06-18,2020-06-08,NaT,31,MÃ©xico,99,TABASCO,CENTRO,TABASCO,TABASCO,NO APLICA,NO,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,SI,SI,NO ESPECIFICADO,NO APLICA,FUERA DE USMER,ESTATAL,HOMBRE,AMBULATORIO,MEXICANA,Positivo SARS-CoV-2,False,NaN,NaN,6,2020
2,2020-09-19,10d9be,2020-04-06,2020-03-28,2020-04-12,43,MÃ©xico,99,BAJA CALIFORNIA,TIJUANA,BAJA CALIFORNIA,DURANGO,SI,SI,NO APLICA,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO ESPECIFICADO,NO ESPECIFICADO,NO,FUERA DE USMER,IMSS,HOMBRE,HOSPITALIZADO,MEXICANA,Positivo SARS-CoV-2,True,4.0,2020.0,4,2020


In [118]:

#Genera error. Solo muestra los encabezados sin datos.
data.query('hipertension == "SI" & defuncion == "True"').head()



/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso


In [136]:

#Genera error. Solo muestra los encabezados sin datos. Hay un error en el registro de hipertension en mi tabla y data set.
data[data.hipertension == 'SI'].head(2)

,fecha_actualizacion,id_registro,fecha_ingreso,fecha_sintomas,fecha_def,edad,pais_nacionalidad,pais_origen,entidad_res,municipio_res,entidad_um,entidad_nac,intubado,neumonia,embarazo,habla_lengua_indig,diabetes,epoc,asma,inmusupr,hipertension,otra_com,cardiovascular,obesidad,renal_cronica,tabaquismo,otro_caso,migrante,uci,origen,sector,sexo,tipo_paciente,nacionalidad,resultado,defuncion,mes_def,año_def,mes_ingreso,año_ingreso


In [137]:
data[data.defuncion].mes_def.value_counts().sort_index()

1.0        6
2.0       12
3.0      232
4.0     5300
5.0    16113
6.0    21546
7.0    24055
8.0    19058
9.0     7298
Name: mes_def, dtype: int64

In [145]:
data[data.tipo_paciente == "HOSPITALIZADO"].mes_def.value_counts().sort_index()

1.0        6
2.0       12
3.0      218
4.0     4872
5.0    14513
6.0    19136
7.0    21270
8.0    17023
9.0     6620
Name: mes_def, dtype: int64

In [151]:
#Respuesta a la pregunta 1.- ¿Tienen los pacientes con hipertensión un riesgo más alto de defunción? 
data.groupby('hipertension').defuncion.value_counts(normalize=True)

hipertension  defuncion
NO            False        0.961119
              True         0.038881
SE IGNORA     False        0.876326
              True         0.123674
SI            False        0.835204
              True         0.164796
Name: defuncion, dtype: float64

In [152]:
data.groupby('diabetes').defuncion.value_counts(normalize=True)

diabetes   defuncion
NO         False        0.958535
           True         0.041465
SE IGNORA  False        0.877358
           True         0.122642
SI         False        0.813853
           True         0.186147
Name: defuncion, dtype: float64

In [153]:
data.groupby('epoc').defuncion.value_counts(normalize=True)

epoc       defuncion
NO         False        0.943377
           True         0.056623
SE IGNORA  False        0.869135
           True         0.130865
SI         False        0.758276
           True         0.241724
Name: defuncion, dtype: float64

In [154]:
data.groupby('asma').defuncion.value_counts(normalize=True)

asma       defuncion
NO         False        0.940440
           True         0.059560
SE IGNORA  False        0.871341
           True         0.128659
SI         False        0.958195
           True         0.041805
Name: defuncion, dtype: float64

In [155]:
data.groupby('inmusupr').defuncion.value_counts(normalize=True)

inmusupr   defuncion
NO         False        0.942115
           True         0.057885
SE IGNORA  False        0.869978
           True         0.130022
SI         False        0.849185
           True         0.150815
Name: defuncion, dtype: float64

In [156]:
data.groupby('otra_com').defuncion.value_counts(normalize=True)

otra_com   defuncion
NO         False        0.943440
           True         0.056560
SE IGNORA  False        0.857312
           True         0.142688
SI         False        0.847849
           True         0.152151
Name: defuncion, dtype: float64

In [157]:
data.groupby('cardiovascular').defuncion.value_counts(normalize=True)

cardiovascular  defuncion
NO              False        0.943408
                True         0.056592
SE IGNORA       False        0.865398
                True         0.134602
SI              False        0.816374
                True         0.183626
Name: defuncion, dtype: float64

In [158]:
data.groupby('obesidad').defuncion.value_counts(normalize=True)

obesidad   defuncion
NO         False        0.946390
           True         0.053610
SE IGNORA  False        0.875089
           True         0.124911
SI         False        0.911034
           True         0.088966
Name: defuncion, dtype: float64

In [159]:
data.groupby('tabaquismo').defuncion.value_counts(normalize=True)

tabaquismo  defuncion
NO          False        0.941129
            True         0.058871
SE IGNORA   False        0.872614
            True         0.127386
SI          False        0.939020
            True         0.060980
Name: defuncion, dtype: float64

In [160]:
data.groupby('otro_caso').defuncion.value_counts(normalize=True)

otro_caso        defuncion
NO               False        0.933622
                 True         0.066378
NO ESPECIFICADO  False        0.807801
                 True         0.192199
SI               False        0.983464
                 True         0.016536
Name: defuncion, dtype: float64

In [162]:
#Respuesta a la pregunta 2.- ¿Cuántos casos confirmados se tienen por Estado?

data[data.resultado == "Positivo SARS-CoV-2"].entidad_res.value_counts().sort_index()



AGUASCALIENTES                       6581
BAJA CALIFORNIA                     18501
BAJA CALIFORNIA SUR                  9374
CAMPECHE                             5851
CHIAPAS                              6427
CHIHUAHUA                            9464
CIUDAD DE MÉXICO                   116943
COAHUILA DE ZARAGOZA                24766
COLIMA                               4380
DURANGO                              8098
GUANAJUATO                          38340
GUERRERO                            17335
HIDALGO                             11930
JALISCO                             24400
MICHOACÁN DE OCAMPO                 18684
MORELOS                              5699
MÉXICO                              76784
NAYARIT                              5642
NUEVO LEÓN                          35910
OAXACA                              15374
PUEBLA                              29938
QUERÉTARO                            8152
QUINTANA ROO                        11290
SAN LUIS POTOSÍ                   

In [146]:
#Respuesta a la pregunta 3 ¿Cuántas defunciones se tienen por estado?

data[data.defuncion].entidad_res.value_counts().sort_index()



AGUASCALIENTES                       662
BAJA CALIFORNIA                     4167
BAJA CALIFORNIA SUR                  559
CAMPECHE                             982
CHIAPAS                             1233
CHIHUAHUA                           2116
CIUDAD DE MÉXICO                   12397
COAHUILA DE ZARAGOZA                2338
COLIMA                               637
DURANGO                              759
GUANAJUATO                          3349
GUERRERO                            2374
HIDALGO                             2180
JALISCO                             4120
MICHOACÁN DE OCAMPO                 2040
MORELOS                             1282
MÉXICO                             14128
NAYARIT                              867
NUEVO LEÓN                          3792
OAXACA                              1666
PUEBLA                              4765
QUERÉTARO                           1056
QUINTANA ROO                        2002
SAN LUIS POTOSÍ                     1924
SINALOA         

In [150]:
#Respuesta a la pregunta 4.- ¿Cuántos fallecimientos han ocurrido en el Estado con mayor número de casos confirmados?

data[data.defuncion].entidad_res.value_counts().sort_index()

estado_mas_defunciones = data[data.entidad].entidad_res.max()  

data[data.entidad_res & (data.defuncion == estado_mas_defunciones)]

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


TypeError: ignored

In [165]:
#Respuesta a la pregunta 4.- ¿Cuántos fallecimientos han ocurrido en el Estado con mayor número de casos confirmados?

# Entiendo el por que sale el resultado. Estamos utilizando como base los registros de datos en la variable (columna) defunción.
# A esta base, la comparamos contra la variable (columna) entidad_res y ejecutamos la función .describe.
# Lo que no entiendo es por que no sale el registro más alto y los otros registros no. 
# ¿Será una cuestión inherente a la forma que está configurada la función?
# Todavía no encuentro una función inversa a .describe ó como invertirla.

data.groupby('defuncion').entidad_res.describe()

,count,unique,top,freq
defuncion,,,,
False,1486757,32,CIUDAD DE MÉXICO,317450
True,93620,32,MÉXICO,14128


In [147]:
#Respuesta a la pregunta 5.- ¿Cuántos fallecimientos han ocurrido en los pacientes Ambulatorios?
data[data.defuncion].tipo_paciente.value_counts().sort_index()

AMBULATORIO       9950
HOSPITALIZADO    83670
Name: tipo_paciente, dtype: int64

In [140]:
#Prueba para validar si la data de hipertensión está correcta y si los errores previos son cuestión de: Syntaxis, data ó la elección de funciones.

data.hipertension.value_counts()

NO           1323409
SI            252537
SE IGNORA       4431
Name: hipertension, dtype: int64

In [141]:
#Prueba para validar si la data de defunciones está correcta y si los errores previos son cuestión de: Syntaxis, data ó la elección de funciones.
data.defuncion.value_counts()

False    1486757
True       93620
Name: defuncion, dtype: int64